# Práctica Minería de Datos

**Este documento contiene la parte práctica de la tarea de evaluación del módulo de Minería de Datos.** 

**La idea es que vayas avanzando en el documento y completando códigos (asegurando ejecutar para que se vean las salidas) y preguntas asociadas.**

**Una vez terminado, lo guardas con el nombre "PracticaMineria_NombreApellido.ipynb" para adjuntarlo a la entrega.** 

**A disfrutar del proceso!**

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import NuestrasFunciones

import plotly.express as px
from plotly.subplots import make_subplots
#from ydata_profiling import ProfileReport
import seaborn as sns

from sklearn.metrics import r2_score
import statsmodels.api as sm

from statsmodels.tsa.seasonal import seasonal_decompose

import warnings
warnings.filterwarnings('ignore')

## Pregunta 1

Indica en el siguiente chunk el código necesario para importar el conjunto de datos **"FEV_data.csv"** y muestra la información del archivo. 

In [3]:
# Lectura del archivo
df_fev = pd.read_csv("FEV_data.csv")
# Información
df_fev.describe()

,seqnbr,subjid,age,fev,height,sex,smoke
count,654.00000,654.000000,654.000000,654.000000,654.000000,654.000000,654.000000
mean,327.50000,37169.571865,9.931193,2.636780,61.143578,1.486239,1.900612
std,188.93782,23690.860350,2.953935,0.867059,5.703513,0.500193,0.299412
min,1.00000,201.000000,3.000000,0.791000,46.000000,1.000000,1.000000
25%,164.25000,15811.000000,8.000000,1.981000,57.000000,1.000000,2.000000
50%,327.50000,36071.000000,10.000000,2.547500,61.500000,1.000000,2.000000
75%,490.75000,53638.500000,12.000000,3.118500,65.500000,2.000000,2.000000
max,654.00000,90001.000000,19.000000,5.793000,74.000000,2.000000,2.000000


¿Cuantos registros y variables tiene el archivo? ¿Tienen las variables el tipo correcto en relación a su lógica naturaleza?

**Respuesta**:


## Pregunta 2

Sabiendo que la variable objetivo es *Fev*. Presenta un gráfico con las relaciones de asociación de los predictores frente a Fev. 

In [2]:
# Gráfico de relaciones con la objetivo


¿Cuales son las 3 variables que elegirías para tu primer modelo de regresión en base a esta información?

**Respuesta**:

## Pregunta 3

Ajusta el modelo de regresión que contiene los 3 predictores que has decidido en el apartado anterior en el propio conjunto de entrenamiento (no hace falta hacer partición) y muestra el summary de dicho modelo. 

In [3]:
# Ajuste de modelo 

# Summary


 A la vista de los valores de ajuste, ¿qué porcentaje de variabilidad explicada tiene el modelo? Interpreta el parámetro estimado de la *altura*.

**Respuesta**:

## Pregunta 4

Aplica validación cruzada repetida para tu modelo con nuestra querida función *cross_val_lin* y una semilla conformada por los 4 últimos dígitos de tu documento de identidad. Muestra el boxplot de sesgo-varianza de las estimaciones. 

In [4]:
# Validación cruzada

# Boxplot sesgo-varianza


A la vista de los resultados. ¿Está sobreajustando tu modelo al conjunto de entrenamiento?

**Respuesta**:

## Pregunta 5

Indica el código necesario para leer el conjunto de datos **"IPI_Esp.xlsx"** y muestra su cabeza.

In [5]:
# Lectura de datos IPI

# Primeras filas


## Pregunta 6

Convierte el conjunto en una serie temporal legible para Python y representa gráficamente la evolución. 

In [6]:
# Conversión a serie temporal legible


In [7]:
# Convertir en date-time


In [8]:
# Mostrar valores de serie


In [9]:
# Gráfico de evolución del IPI


¿Qué componentes de la serie se pueden intuir en el gráfico?

**Respuesta**:

## Pregunta 7

Toma una partición con ventana de entrenamiento (todo el histórico hasta diciembre de 2017) y prueba (2018 y 2019). Ajusta el modelo de suavizado que consideres más adecuado a la luz de las características de la serie. Evalua el modelo en términos de error y adecuación de los residuos. ¿El modelo satisface las condiciones residuales en cuanto a  autocorrelaciones?

In [10]:
# Partición de datos

# Ajuste de modelo de suavizado


**Respuesta**:


## Pregunta 8

Presenta los correlogramas simple y parcial de la serie diferenciada en lo regular y estacional para el conjunto de entrenamiento. 

In [11]:
# Diferenciar serie

# Correlograma simple ACF

# Correlograma parcial PACF


A pesar de la dificultad, intentemos proponer algunos órdenes en lo regular y estcional para un tentativo modelo arima. 

¿Cual sería tu primer modelo a probar según lo que observas en la desaparición de las correlaciones con los retardos?

**Respuesta**: Mi modelo sería ARIMA(p,d,q)(P,D,Q)

## Pregunta 9

Ajusta ese modelo que propones y evalúa su capacidad de ajuste en test y la adecuación de residuos. 

In [12]:
# Ajuste de modelo ARIMA


In [13]:
# Summary del modelo 


¿Son todos los efectos significativos en el modelo?

**Respuesta**: 

In [14]:
# Evaluación del modelo (eval_model)


¿Es tu modelo válido para la predicción en términos de adecuación residual? En caso contrario, juega con los órdenes para obtener un modelo que satisfaga el test de residuos. ¿Es este modelo aceptable mejor que el suavizado en términos de capacidad de predicción en el conjunto de test? 

**Respuesta**: 

## Pregunta 10

Indica el código necesario para leer el archivo **"clientes_cluster.csv"** y mostrar sus primeros registros. 

In [15]:
# Lectura clientes_cluster


## Pregunta 11

Evalúa la incidencia de valores perdidos por variable e imputa el archivo mediante el método multivariante basado en el algoritmo KNN, generando un nuevo archivo completo **"clientes_imputed"** del que se mostrará la información para comprobar que efectivamente todos los registros contienen valores válidos.

In [16]:
#Incidencia de missings por variable


**Comentario**:

In [17]:
# Imputación por KNN

# Info del archivo imputado


Recodifica *Gender* como una variable dummy (cualquier opción de código es bienvenida) en formato numérico y elimina el ID de cliente de cara al análisis.

In [18]:
# Recodificación de Gender como dummy


## Pregunta 12

Explora los métodos de clustering jerárquico para formar grupos de clientes similares en cuanto a su comportamiento en estas 4 variables disponibles y presenta sus dendogramas. Si lo consideras necesario, escala los datos previamente.

In [19]:
# Clustering jerárquico bucle de dendogramas
    

¿Cuál es el método de linkage más adecuado a la vista de los dendogramas? ¿Cuantos grupos de clientes formarías en base a las alturas del dendograma del linkage escogido?

**Respuesta**:

## Pregunta 13

Explora el clustering K-means con distinto número de grupos y decide cuantos el número de clusters óptimo para la solución según este método. Comenta las métricas de evaluación.

In [20]:
# Clustering k-means evaluación número clusters


**Respuesta**: 

Ajusta el clustering K-means con el núemero de grupos seleccionado y genera una nueva variable en el conjunto de datos imputado llamada *'Cluster'* que recoja el número de grupo asignado a cada registro. El tipo de la variable debe ser entero (int).  

In [21]:
# Ajuste de k-means

# Creación de variable cluster


Extrae los centroides de los grupos y comenta las principales características de cada grupo formado. 

In [22]:
# Centroides


**Comentarios**: 

## Pregunta 14

De cara a la proyección de los grupos formados en el espacio de características y dado que estamos en R4. Nos planteamos reducir la dimensionalidad a 2 para poder interpretar con mayor claridad mediante un bonito biplot. 

Para llevar a cabo esta tarea, necesitaremos ajustar un PCA con 2 dimensiones a los datos de entrada al clustering que hemos generado, es decir, tal vez utilizamos el conjunto de datos escalado si lo consideramos necesario. Mostraremos las cargas de las variables en las componentes y haremos una pequeña interpretación inicial de las 2 componentes.


In [23]:
# Ajuste PCA 2 dimensiones

# Análisis de cargas

**Interpretación inicial**:

Ahora generamos y mostramos el biplot para interpretar con mayor seguridad las componentes. 

In [24]:
# Biplot


**Interpretación gráfica**:

Ahora que sabemos lo que represanta cada componente en el espacio original de las variables, necesitamos un conjunto de datos que contenga las columnas *PC1*,*PC2* y *Cluster* para realizar la representación de los clientes por grupos propuestos e interpretar las características de los grupos en el espacio de las componentes. 

In [25]:
# Creación dataset dimensión reducida

# Representación de grupos en espacio de componentes


**Interpretación de biplot**: